In [2]:
import numpy as np
import pandas as pd
from sbfl.base import SBFL, Ensemble
from scipy.stats import rankdata

#   e1,e2,e3,e4,e5,e6
X = np.array([
    [1,1,0,0,1,1], # coverage of test t0
    [1,1,0,0,1,1], # coverage of test t1
    [0,1,1,0,1,0], # coverage of test t2
    [1,0,0,1,0,0],
    [0,1,0,1,1,0],
], dtype=bool)

y = np.array([1,1,0,0,0], dtype=bool)

X, y

(array([[ True,  True, False, False,  True,  True],
        [ True,  True, False, False,  True,  True],
        [False,  True,  True, False,  True, False],
        [ True, False, False,  True, False, False],
        [False,  True, False,  True,  True, False]]),
 array([ True,  True, False, False, False]))

In [3]:
sbfl = SBFL(formula='Ochiai')
sbfl.fit_predict(X, y)

array([0.33333333, 0.57735027, 0.57735027, 0.81649658, 0.57735027,
       0.        ])

In [4]:
sbfl.ranks(method='max')

array([5, 4, 4, 1, 4, 6])

In [5]:
names = ['file', 'method', 'line']
elements = [
    ('file1.py', 'method1', 'line1'),
    ('file2.py', 'method1', 'line1'),
    ('file2.py', 'method1', 'line2'),
    ('file2.py', 'method2', 'line3'),
    ('file2.py', 'method2', 'line4'),
    ('file2.py', 'method3', 'line5'),
]
score_df = sbfl.to_frame(elements=elements, names=names)
score_df

score
file     method  line           
file1.py method1 line1  0.333333
file2.py method1 line1  0.577350
                 line2  0.577350
         method2 line3  0.816497
                 line4  0.577350
         method3 line5  0.000000

In [6]:
from sbfl.score_aggregation import dense_rank_based_suspiciousness_aware_voting

dense_rank_based_suspiciousness_aware_voting(score_df, ['file', 'method'])

score
file     method           
file1.py method1  0.111111
file2.py method1  0.577350
         method2  1.105172
         method3  0.000000

# Ensemble

In [7]:
# single model
sbfl = SBFL(formula='Ochiai')
sbfl.fit_predict(X, y)

array([0.33333333, 0.57735027, 0.57735027, 0.81649658, 0.57735027,
       0.        ])

In [8]:
# multiple models
en = Ensemble()
en.add_localizer(SBFL(formula="Ochiai"))
en.add_localizer(SBFL(formula="Op2"))
en.fit_predict(X, y)

array([0.4       , 0.58333333, 0.5       , 2.        , 0.58333333,
       0.33333333])

# Read GCOV files

In [9]:
gcov_dir = {
    'test100': 'resources/yara-buggy#3-100',
    'test101': 'resources/yara-buggy#3-101',
    'test102': 'resources/yara-buggy#3-102',
    'test103': 'resources/yara-buggy#3-103',
    'test104': 'resources/yara-buggy#3-104',
}
gcov_files = {test:[] for test in gcov_dir}
for test in gcov_dir:
    for path in os.listdir(gcov_dir[test]):
        if path.endswith('.gcov'):
            gcov_files[test].append(os.path.join(gcov_dir[test], path))
    print(f"{test}: {len(gcov_files[test])} gcov files are found.")

test100: 48 gcov files are found.
test101: 48 gcov files are found.
test102: 48 gcov files are found.
test103: 48 gcov files are found.
test104: 48 gcov files are found.


In [10]:
from sbfl.utils import gcov_files_to_frame, get_sbfl_scores_from_frame

cov_df = gcov_files_to_frame(gcov_files, only_covered=True, verbose=True)
cov_df

100%|██████████| 5/5 [00:01<00:00,  3.90it/s]


test100  \
file                                               function line            
/home/workspace/libyara/threading.gcno,threading.c NaN      172        12   
                                                            175        12   
                                                            178        12   
                                                            182         8   
                                                            185         8   
...                                                                   ...   
/home/workspace/libyara/hex_lexer.gcno,hex_lexer.l NaN      252         1   
                                                            253         1   
                                                            254         1   
                                                            256         1   
                                                            262         1   

                                                                  test101  \
file                                               function line            
/home/workspace/libyara/threading.gcno,threading.c NaN      172        12   
                                                            175        12   
                                                            178        12   
                                                            182         8   
                                                            185         8   
...                                                                   ...   
/home/workspace/libyara/hex_lexer.gcno,hex_lexer.l NaN      252         1   
                                                            253         1   
                                                            254         1   
                                                            256         1   
                                                            262         1   

                                                                  test102  \
file                                               function line            
/home/workspace/libyara/threading.gcno,threading.c NaN      172        12   
                                                            175        12   
                                                            178        12   
                                                            182         8   
                                                            185         8   
...                                                                   ...   
/home/workspace/libyara/hex_lexer.gcno,hex_lexer.l NaN      252         1   
                                                            253         1   
                                                            254         1   
                                                            256         1   
                                                            262         1   

                                                                  test103  \
file                                               function line            
/home/workspace/libyara/threading.gcno,threading.c NaN      172        12   
                                                            175        12   
                                                            178        12   
                                                            182         8   
                                                            185         8   
...                                                                   ...   
/home/workspace/libyara/hex_lexer.gcno,hex_lexer.l NaN      252         0   
                                                            253         0   
                                                            254         0   
                                                            256         0   
                                                            262         0   

                                                          

In [11]:
score_df = get_sbfl_scores_from_frame(cov_df, failing_tests=['test102'])

In [12]:
dense_rank_based_suspiciousness_aware_voting(score_df, ['file'])

,score
file,
"/home/workspace/libyara/atoms.gcno,atoms.c",42.570357
"/home/workspace/libyara/bitmask.gcno,bitmask.c",0.000000
"/home/workspace/libyara/hex_grammar.gcno,hex_grammar.c",17.897858
"/home/workspace/libyara/hex_grammar.gcno,hex_grammar.y",5.070765
"/home/workspace/libyara/hex_lexer.gcno,hex_lexer.c",37.720218
"/home/workspace/libyara/hex_lexer.gcno,hex_lexer.l",5.102112
"/home/workspace/libyara/libyara.gcno,libyara.c",4.360333
"/home/workspace/libyara/mem.gcno,mem.c",1.006231
"/home/workspace/libyara/modules.gcno,modules.c",1.118034
